In [1]:
import pandas as pd
import json 
import glob

In [2]:
list_ = []
for i in glob.glob('*.geojson'):
    list_.append(i)
df = pd.read_json(list_[0])
df = pd.json_normalize(df['features'])

In [3]:
list_2 = []
for i in list_:
    if i == 1:
        continue
    data = pd.read_json(i)
    dataa = pd.json_normalize(data['features'])
    list_2.append(dataa)
df = pd.concat(list_2)

In [4]:
df2 = df.copy()
df2.drop(['type', 'geometry.type', 'geometry.coordinates', 'properties.id', 'properties.point.lat', 'properties.point.long',
        'properties.nearby', 'properties.scheme', 'properties.datetime', 'properties.address'], axis=1, inplace=True)
df2.head(2)

,properties.tags,properties.light,properties.region,properties.weather,properties.category,properties.severity,properties.vehicles,properties.dead_count,properties.participants,properties.injured_count,properties.parent_region,properties.road_conditions,properties.participants_count,properties.participant_categories
0,[Дорожно-транспортные происшествия],Светлое время суток,Барнаул,[Ясно],Столкновение,Легкий,"[{'year': 1998, 'brand': 'SUBARU', 'color': 'Б...",0,[],4,Алтайский край,[Сухое],5,[Все участники]
1,[Дорожно-транспортные происшествия],Светлое время суток,Хабарский район,[Ясно],Наезд на пешехода,Легкий,"[{'year': 2006, 'brand': 'TOYOTA', 'color': 'С...",0,"[{'role': 'Пешеход', 'gender': 'Женский', 'vio...",1,Алтайский край,[Со снежным накатом],2,"[Пешеходы, Все участники]"


In [5]:
df2['properties.road_conditions'] = df2['properties.road_conditions'].apply(lambda x: ''.join(x))
df2['properties.tags'] = df2['properties.tags'].apply(lambda x: ''.join(x))
df2['properties.weather'] = df2['properties.weather'].apply(lambda x: ''.join(x))
df2['properties.participant_categories'] = df2['properties.participant_categories'].apply(lambda x: ''.join(x))
df2.head(2)

,properties.tags,properties.light,properties.region,properties.weather,properties.category,properties.severity,properties.vehicles,properties.dead_count,properties.participants,properties.injured_count,properties.parent_region,properties.road_conditions,properties.participants_count,properties.participant_categories
0,Дорожно-транспортные происшествия,Светлое время суток,Барнаул,Ясно,Столкновение,Легкий,"[{'year': 1998, 'brand': 'SUBARU', 'color': 'Б...",0,[],4,Алтайский край,Сухое,5,Все участники
1,Дорожно-транспортные происшествия,Светлое время суток,Хабарский район,Ясно,Наезд на пешехода,Легкий,"[{'year': 2006, 'brand': 'TOYOTA', 'color': 'С...",0,"[{'role': 'Пешеход', 'gender': 'Женский', 'vio...",1,Алтайский край,Со снежным накатом,2,ПешеходыВсе участники


In [13]:
df3 = df2.copy()
df3.drop(['properties.tags', 'properties.region', 'properties.vehicles', 'properties.participants', 
               'properties.parent_region', 'properties.participant_categories'], axis=1, inplace=True)
df3.head(2)

,properties.light,properties.weather,properties.category,properties.severity,properties.dead_count,properties.injured_count,properties.road_conditions,properties.participants_count
0,Светлое время суток,Ясно,Столкновение,Легкий,0,4,Сухое,5
1,Светлое время суток,Ясно,Наезд на пешехода,Легкий,0,1,Со снежным накатом,2


In [16]:
df3.isna().sum()

properties.light                 0
properties.weather               0
properties.category              0
properties.severity              0
properties.dead_count            0
properties.injured_count         0
properties.road_conditions       0
properties.participants_count    0
dtype: int64

### Кодируем категориальные столбцы  

In [17]:
df4 = df3.copy()
from sklearn.preprocessing import LabelEncoder

In [20]:
light = LabelEncoder()
df4['properties.light'] = light.fit_transform(df4['properties.light'])

weather = LabelEncoder()
df4['properties.weather'] = weather.fit_transform(df4['properties.weather'])

severity = LabelEncoder()
df4['properties.severity'] = severity.fit_transform(df4['properties.severity'])

category = LabelEncoder()
df4['properties.category'] = category.fit_transform(df4['properties.category'])

road_conditions = LabelEncoder()
df4['properties.road_conditions'] = road_conditions.fit_transform(df4['properties.road_conditions'])

### Кластеризация 

#### K-means

In [23]:
from sklearn.cluster import KMeans
model = KMeans(n_clusters=2, n_init='k-means++')
y_pred = model.fit_predict(df4)
pd.DataFrame(y_pred)[0].value_counts()

TypeError: '<=' not supported between instances of 'str' and 'int'

In [22]:
df3['y'] = y_pred
df3

,properties.light,properties.weather,properties.category,properties.severity,properties.dead_count,properties.injured_count,properties.road_conditions,properties.participants_count,y
0,Светлое время суток,Ясно,Столкновение,Легкий,0,4,Сухое,5,0
1,Светлое время суток,Ясно,Наезд на пешехода,Легкий,0,1,Со снежным накатом,2,0
2,Светлое время суток,Ясно,Столкновение,Тяжёлый,0,3,Сухое,4,0
3,Светлое время суток,Ясно,Опрокидывание,Тяжёлый,0,1,Сухое,2,0
4,"В темное время суток, освещение отсутствует",Ясно,Наезд на пешехода,С погибшими,1,0,Мокрое,2,1
...,...,...,...,...,...,...,...,...,...
3140,Светлое время суток,Ясно,Наезд на велосипедиста,Легкий,0,1,Сухое,2,0
3141,Сумерки,Ясно,Наезд на препятствие,Легкий,0,1,Сухое,5,0
3142,Светлое время суток,Ясно,Наезд на препятствие,Легкий,0,1,Недостатки зимнего содержанияСо снежным накатом,1,1
3143,Светлое время суток,Ясно,Наезд на препятствие,Тяжёлый,0,1,Сухое,1,0


In [11]:
df.to_csv('dataframe.csv', index=False)